# 1 Giriş

Herkese Merhaba, Burada, en büyük Rus yazılım firmalarından biri olan *1C Company* tarafından sağlanan günlük satış verilerinden oluşan zorlu bir zaman serisi veri kümesiyle çalışacağız.Her ürün ve mağaza için toplam satışları tahmin edeceğiz. 

Bu Notebook ile, veri bilimi becerilerinizi uygulayabilir ve geliştirebilirsiniz. Ve tabi belki yeni bir şeyler öğrenirsiniz.

Sevgilerimle, Başlayalım

In [ ]:
import numpy as np 
import pandas as pd
import datetime
import os
import seaborn as sns
import matplotlib.pylab as plt

# 2 Verileri Yükleme

In [ ]:

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
dfShops = pd.read_csv('/kaggle/input/predict-future-sales-translated-dataset/shops_en.csv')
dfItems = pd.read_csv('/kaggle/input/predict-future-sales-translated-dataset/items_en.csv')
dfCateg = pd.read_csv('/kaggle/input/predict-future-sales-translated-dataset/item_categories_en.csv')
dfSales = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
dfSTest = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
dfSubm  = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv')

# 3 Verileri Analiz Etme

In [ ]:
dfSTest.head()

In [ ]:
dfShops.head()

In [ ]:
dfCateg.head()

In [ ]:
dfSales.head()

Kolonların ne anlama geldiği aşağıda verilmiştir.


* **item_cnt_day**: satılan ürünlerin sayısı. Bu önlemin aylık miktarını tahmin edeceğiz.
* **shop_id** : bir mağazanın benzersiz tanımlayıcısı
* **item_id** : bir ürünün benzersiz tanımlayıcısı
* **tarih_block_num** : kolaylık sağlamak için kullanılan ardışık bir ay numarası. Ocak 2013 0, Şubat 2013 1, ... Ekim 2015 33
* **item_price** : ürünün fiyatı

In [ ]:
# Mağazaların dağılımı 
dfSales.shop_id.value_counts()

In [ ]:
# Ürünlerin dağılımı 
dfSales.item_id.value_counts()

dfSales ile çalışacağım ve bir Time Series Analizi yapacağım. Bunun için öncelikle en çok satışı olan mağazanın en çok satan ürünü ile veriyi kısıtlayacağım.
Ve o ürünün o mağazadaki satışını tahmin edeceğim. İlk olarak mağaza ile kısıtlıyorum.

# 4 Veri Seti Oluşturma

In [ ]:
dfShopId31=dfSales.loc[dfSales.shop_id==31,:].reset_index(drop=True)
dfShopId31.shape

In [ ]:
dfShopId31.head()

Şimdi de ürün bazlı kısıtlamaya tabi tutuyorum.

In [ ]:
dfShopId31.item_id.value_counts()

In [ ]:
dfShopId31item20949=dfShopId31.loc[dfShopId31.item_id==20949,:].reset_index(drop=True)
dfShopId31item20949.head()

Son olarak date kolonunu tarihsel olarak sıralıyorum. 

In [ ]:
dfShopId31item20949.dtypes

Öncelikle datetime formatına çeviriyorum.

In [ ]:
dfShopId31item20949["date"]=dfShopId31item20949["date"].apply(lambda x: x.replace(".","-"))
dfShopId31item20949["date"]=dfShopId31item20949["date"].apply(lambda x: datetime.datetime.strptime(x, '%d-%m-%Y'))
dfShopId31item20949.head()

In [ ]:
dfShopId31item20949.dtypes

Görüldüğü gibi datetime formatına çevirdim. Devam ediyorum.

In [ ]:
dfShopId31item20949=dfShopId31item20949.sort_values("date")
dfShopId31item20949.head()

Ben bu *date_block_num* olayını sevmedim. O yüzden date kolonunu time series'e uyarlamak için ay ve yıldan oluşan bir kolon oluşturacağım.
Sonradan gruplama yapacağım.

In [ ]:
dfShopId31item20949["DMonth"]=dfShopId31item20949["date"].apply(lambda x: x.strftime("%m"))
dfShopId31item20949["DYear"]=dfShopId31item20949["date"].apply(lambda x: x.strftime("%Y"))
dfShopId31item20949["DateCol"]="01-"+dfShopId31item20949["DMonth"]+"-"+dfShopId31item20949["DYear"]
dfShopId31item20949["DateCol"]=dfShopId31item20949["DateCol"].apply(lambda x: datetime.datetime.strptime(x, '%d-%m-%Y'))
dfShopId31item20949.head()

In [ ]:
dfShopId31item20949.dtypes

*DateCol* kolonunun datetime olduğunu gördüğümüze göre devam edelim. 
Gruplama yapacağım.

In [ ]:
TargetDF=dfShopId31item20949.groupby("DateCol").item_cnt_day.sum().reset_index()
TargetDF

# 5 Veri Görselleştirme

In [ ]:
plt.figure(figsize=(20, 6))
sns.lineplot(data=TargetDF, x="DateCol", y="item_cnt_day")